In [1]:
%load_ext autoreload
%autoreload 2
import torch
import wandb
from dataset.dataset import ImageDataset
from dataset.normalization import MinMax
from dataset.filters import sza_filter_95
from models.ConvResNet_Jiang import ConvResNet
from models.LightningModule import LitEstimator, LitEstimatorPoint
from lightning.pytorch import Trainer
from lightning.pytorch.loggers import WandbLogger
from lightning.pytorch.plugins.environments import SLURMEnvironment
from lightning.pytorch.utilities import rank_zero_only
from tqdm import tqdm

# from pytorch_lightning.pytorch.callbacks import DeviceStatsMonitor
from utils.plotting import best_worst_plot, prediction_error_plot




In [2]:
from types import SimpleNamespace
config = {
    "batch_size": 4096,
    "patch_size": {'x':15, 
                   'y':15,
                   'stride_x':1,
                   'stride_y':1,
                  },
    "x_vars": [
        "channel_1",
        "channel_2",
        "channel_3",
        "channel_4",
        "channel_5",
        "channel_6",
        "channel_7",
        "channel_8",
        "channel_9",
        "channel_10",
        "channel_11",
        "DEM"
    ],
    "y_vars": ["SIS", "CAL", "SID"],
    "x_features": ["dayofyear", "lat", "lon", "SZA", "AZI"],
    "transform": MinMax(),
    "target_transform": MinMax(),
}
config = SimpleNamespace(**config)

In [4]:
dataset = ImageDataset(
    x_vars=config.x_vars,
    y_vars=config.y_vars,
    x_features=config.x_features,
    patch_size=config.patch_size,
    transform=config.transform,
    target_transform=config.target_transform,
)

model = ConvResNet(
    num_attr=len(config.x_features), 
    input_channels=len(config.x_vars),
    output_channels=len(config.y_vars)
    )

dataloader= torch.utils.data.DataLoader(dataset, batch_size=config.batch_size,
                        shuffle=False, num_workers=0)
    


pre dropna 35966
dropna 18348
pre dropna 35040
dropna 10111


In [5]:
# Compute related
ACCELERATOR = "gpu"
DEVICES = -1
NUM_NODES = 1
# STRATEGY = "ddp"
PRECISION = 32

trainer = Trainer(
    # profiler="simple",
    # fast_dev_run=True,
    num_sanity_val_steps=2,
    # logger=wandb_logger,
    accelerator=ACCELERATOR,
    devices=DEVICES,
    # num_nodes=NUM_NODES,
    # strategy=STRATEGY,
    min_epochs=1,
    max_epochs=1,
    precision=PRECISION,
    log_every_n_steps=200,
    # val_check_interval=.5,
    # plugins=[SLURMEnvironment(auto_requeue=False)],
)

/scratch/snx3000/kschuurm/pytorch/lib/python3.9/site-packages/lightning/fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /scratch/snx3000/kschuurm/pytorch/lib/python3.9/site ...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [6]:
estimator = LitEstimatorPoint(
            model=model,
            learning_rate=0.001,
        )


trainer.fit(model=estimator, train_dataloaders=dataloader)


/scratch/snx3000/kschuurm/pytorch/lib/python3.9/site-packages/lightning/pytorch/utilities/parsing.py:198: Attribute 'model' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['model'])`.
/scratch/snx3000/kschuurm/pytorch/lib/python3.9/site-packages/lightning/fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /scratch/snx3000/kschuurm/pytorch/lib/python3.9/site ...
/scratch/snx3000/kschuurm/pytorch/lib/python3.9/site-packages/lightning/pytorch/trainer/configuration_validator.py:74: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type             | Params
---------------------------------------------------
0 | model         | ConvRe

Training: |          | 0/? [00:00<?, ?it/s]

new image
new image
new image
new image
new image
new image
new image
new image
new image
new image
new image
new image


/scratch/snx3000/kschuurm/pytorch/lib/python3.9/site-packages/lightning/pytorch/trainer/call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...
